In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
include("ETO.jl")
include("RO.jl")

Solve_RO_Price_with_one_side_exp (generic function with 1 method)

### Parameters

In [2]:
function generate_strictly_row_diagonally_dominant(n::Int, max_offdiag,offdiag_sign)
    Mat = zeros(n, n)
    for i in 1:n
        # 在非对角线上生成随机数 [-max_offdiag, max_offdiag]
        off_diag = rand(n) .* (2max_offdiag) .- max_offdiag
        off_diag[i] = 0  # 避免给自己赋值两次

        # 计算非对角元素绝对值之和
        sum_offdiag = sum(abs, off_diag)

        # 设置对角元，使其严格大于其他元素之和
        diag_value = sum_offdiag + rand() * max_offdiag + 1e-3  # 加小量保证严格性

        if offdiag_sign == "positive" 
            Mat[i, :] = abs.(off_diag)
        end
        if offdiag_sign == "negative" 
            Mat[i, :] = -abs.(off_diag)
        end
        if offdiag_sign == "mix" 
            Mat[i, :] = off_diag
        end
        if offdiag_sign == "zero" 
            Mat[i, :] = off_diag .* 0.0
        end        
        Mat[i, i] = -abs.(diag_value)
    end
    return Mat
end

generate_strictly_row_diagonally_dominant (generic function with 1 method)

In [3]:
function compute_oof(X_given, A, B, u, p_dag)
    price = sum(X_given .* p_dag,dims=2)[:,1]
    utilities = ones(N+1)
    utilities[1:N] = exp.(vec(A * u + B * price))

    prob = zeros(N)
    rev = zeros(N)
    for i in 1:N
        prob[i] = utilities[i] / sum(utilities)
        rev[i] = prob[i] * price[i]
    end
    # println("prob = ", prob)
    # println("rev = ", rev)
    # println("total rev = ", sum(rev))
    return sum(rev),price
end

compute_oof (generic function with 1 method)

### Iteratively search for the optimal solution

In [ ]:
# rev_opt = 0
# price_opt = zeros(N)

# rev_min = 1000
# price_min = zeros(N)
# for i in 1:K
#     p1 = p_dag[1,i]
#     for j in 1:K
#         p2 = p_dag[2,j]
#         for l in 1:K
#             p3 = p_dag[3,l]
#             price = [p1,p2,p3]
#             utilities = ones(N+1)
#             utilities[1:N] = exp.(vec(A * u + B * price))
        
#             prob = zeros(N)
#             rev = zeros(N)
#             for i in 1:N
#                 prob[i] = utilities[i] / sum(utilities)
#                 rev[i] = prob[i] * price[i]
#             end
#             if sum(rev) > rev_opt
#                 rev_opt = sum(rev)
#                 price_opt = price 
#             end

#             if sum(rev) < rev_min
#                 rev_min = sum(rev)
#                 price_min = price
#             end
#         end
#     end
# end
# println("Optimal revenue = ", rev_opt)
# println("Optimal price = ", price_opt)

# println("Minimum revenue = ", rev_min) 
# println("Minimum price = ", price_min)

In [4]:
function Solve_RO_Price(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub,phi_lb,phi_ub)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, delta)                           # 标量 δ
    @variable(model, omega_lb[1:N,1:N_u] >= 0)            
    @variable(model, omega_ub[1:N,1:N_u] >= 0)           
    @variable(model, pi_lb[1:N,1:N] >= 0)     
    @variable(model, pi_ub[1:N,1:N] >= 0)     
    @variable(model, eta_lb[1:N,1:N_u] >= 0)            
    @variable(model, eta_ub[1:N,1:N_u] >= 0)           
    @variable(model, ups_lb[1:N,1:N] >= 0)     
    @variable(model, ups_ub[1:N,1:N] >= 0) 
    # exponential variables
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    @variable(model, phi_1[1:N])                   
    @variable(model, phi_2[1:N])                   
    @variable(model, phi_3[1:N])         

    @variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N,1:K] <= 0)    
    @variable(model, Z[1:N,1:N,1:K] <= 0)    

    for n in 1:N
        @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
    end

    for n in 1:N
        @constraint(model, eta_lb[n,:] .- eta_ub[n,:] .- phi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, ups_lb[n,:] .- ups_ub[n,:] .- sum(Z[n,:,:] .* p_dag,dims=2) .== 0)
    end

    @constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) + sum(phi_1) <= 0)

    for n in 1:N
        @constraint(model, eta_ub[n,:]' * A_ub[n,:] - eta_lb[n,:]' * A_lb[n,:] + ups_ub[n,:]' * B_ub[n,:] - ups_lb[n,:]' * B_lb[n,:] + delta + psi_1[n] + phi_2[n] - X[n,:]' * p_dag[n,:] <= 0)
    end

    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end

    for n in 1:N
        @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.ExponentialCone())
    end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
                @constraint(model, Y[n,j,k] <= psi_ub * X[j,k])
                @constraint(model, Y[n,j,k] >= psi_3[n] - psi_ub * (1 - X[j,k]))
                @constraint(model, Y[n,j,k] <= psi_3[n] - psi_lb * (1 - X[j,k]))
            end
        end
    end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Z[n,j,k] >= phi_lb * X[j,k])
                @constraint(model, Z[n,j,k] <= phi_ub * X[j,k])
                @constraint(model, Z[n,j,k] >= phi_3[n] - phi_ub * (1 - X[j,k]))
                @constraint(model, Z[n,j,k] <= phi_3[n] - phi_lb * (1 - X[j,k]))
            end
        end
    end

    @constraint(model, sum(X,dims=2) .== 1)

    for n in 1:N
        for j in 1:N
            @constraint(model, sum(Y[n,j,:]) == psi_3[n])
            @constraint(model, sum(Z[n,j,:]) == phi_3[n])
        end
    end


    @objective(model, Max, delta)

    optimize!(model)
    status = JuMP.termination_status(model)
    # println("status: ", status)
    # solution_summary(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        X_val = round.(value.(X))
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val,X_val,solve_time
end

Solve_RO_Price (generic function with 1 method)

In [5]:
function Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, delta)                           # 标量 δ
    @variable(model, omega_lb[1:N,1:N_u] >= 0)            
    @variable(model, omega_ub[1:N,1:N_u] >= 0)           
    @variable(model, pi_lb[1:N,1:N] >= 0)     
    @variable(model, pi_ub[1:N,1:N] >= 0)     

    # exponential variables
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    
    @variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N,1:K] <= 0)    

    for n in 1:N
        @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
    end

    for n in 1:N
        @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
    end

    @constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) <= 0)

    for n in 1:N
        @constraint(model, delta + psi_1[n] - X[n,:]' * p_dag[n,:] <= 0)
    end

    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end


    # for n in 1:N
    #     for j in 1:N
    #         for k in 1:K
    #             @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
    #             @constraint(model, Y[n,j,k] <= psi_ub * X[j,k])
    #             @constraint(model, Y[n,j,k] >= psi_3[n] - psi_ub * (1 - X[j,k]))
    #             @constraint(model, Y[n,j,k] <= psi_3[n] - psi_lb * (1 - X[j,k]))
    #         end
    #     end
    # end

    for n in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(model, Y[n,j,k] >= psi_lb * X[j,k])
                @constraint(model, Y[n,j,k] <= 0.0)
            end
        end
    end    
    for n in 1:N
        for j in 1:N
            @constraint(model, sum(Y[n,j,:]) == psi_3[n])
        end
    end

    @constraint(model, sum(X,dims=2) .== 1)
    @objective(model, Max, delta)

    optimize!(model)
    status = JuMP.termination_status(model)
    # println("status: ", status)
    # solution_summary(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        X_val = round.(value.(X))
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val,X_val,solve_time
end

Solve_RO_Price_with_one_side_exp (generic function with 1 method)

## Evaluate

### Parameters

In [6]:
Random.seed!(1)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "negative"
# 预设数据（根据你的实际问题替换这些值）
u = rand(N_u)        # u 向量 ∈ ℝ^N
A_true = rand(N,N_u)      # a_n ∈ ℝ^N
# B = rand(N,N)        # b_n ∈ ℝ^N
p_dag = round.(rand(N, K),digits=2)   # p_{nk}^† 数据矩阵
B_true = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)

5×5 Matrix{Float64}:
 -2.9801    -0.593663  -0.911183   -0.188222  -0.446405
 -0.854162  -3.17998   -0.965216   -0.596854  -0.280659
 -0.401324  -0.372139  -2.4523     -0.498861  -0.311921
 -0.507205  -0.148742  -0.0190967  -2.16626   -0.809852
 -0.639684  -0.890079  -0.175896   -0.958823  -3.64719

In [7]:
bd_coef = 0.00001
A_lb = A_true .- bd_coef .* abs.(rand(N,N_u))
A_ub = A_true .+ bd_coef .* abs.(rand(N,N_u))
B_lb = B_true .- bd_coef .* abs.(rand(N,N))
B_ub = B_true .+ bd_coef .* abs.(rand(N,N))
# obj_RO,X_RO = Solve_RO(N,N_u,K,tau,A_lb,A_ub,B_lb,B_ub,u,p_dag);
# total_rev = compute_oof(X_RO, A, B, u, p_dag)
# println("total rev: ", total_rev) 
# price_RO = sum(X_RO .* p_dag,dims=2)
# println("price_RO: ", price_RO) 

5×5 Matrix{Float64}:
 -2.98009   -0.593654  -0.91118   -0.188218  -0.446398
 -0.854153  -3.17997   -0.965207  -0.596851  -0.280659
 -0.40132   -0.372134  -2.4523    -0.498851  -0.311919
 -0.507202  -0.148733  -0.019092  -2.16625   -0.809848
 -0.639683  -0.890079  -0.175893  -0.958815  -3.64718

### ETO

In [8]:
obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,u,p_dag)
rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, u, p_dag)
println("rev_ETO=",rev_ETO,",price_ETO = ",price_ETO)

rev_ETO=0.20135610001083498,price_ETO = [0.37, 0.45, 0.51, 0.57, 0.28]


### RO with one-side exponential cone

In [ ]:
for i in 1:9
    psi_lb = -1.0 - i * 1.0
    psi_ub = 0.1
    # println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb)
    obj_RO,X_RO,time_RO = Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
    rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
    println("psi_lb = ",psi_lb,",rev_RO = ",rev_RO,",price_RO = ",price_RO)
end

psi_lb = -2.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -3.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -4.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -5.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -6.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -7.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -8.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -9.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]
psi_lb = -10.0,rev_RO = 0.20135610001083498,price_RO = [0.37, 0.45, 0.51, 0.57, 0.28]


In [32]:
p_dag

5×10 Matrix{Float64}:
 0.83  0.89  0.84  0.18  0.24  0.95  0.68  0.37  0.84  0.13
 0.57  0.45  0.65  0.07  0.75  0.2   0.67  0.17  0.68  0.86
 0.18  0.21  0.1   0.72  0.29  0.02  0.66  0.79  0.51  0.97
 0.11  0.19  0.57  0.42  0.62  0.69  0.42  0.33  0.81  0.29
 0.79  0.28  0.78  0.44  0.81  0.8   0.24  0.47  0.81  0.92

### RO with two-side exponential cone

In [ ]:
for i in 1:9
    psi_lb = -0.33 - i * 0.001
    psi_ub = 0.1
    phi_lb = -0.33 - i * 0.001
    phi_ub = 0.1
    # println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb)
    obj_RO,X_RO,time_RO = Solve_RO_Price(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub,phi_lb,phi_ub)
    rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
    println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",rev_RO = ",rev_RO,",price_RO = ",price_RO)
end

### More experiments

In [ ]:
Random.seed!(1)
# 设置参数
N = 3   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2

offdiag_sign = "mix"
iterations = 10
bias_coef = [0.01,0.05,0.1]
ro_coef_all = [0.00001,0.01,0.1,0.15,0.2]

rev_True = zeros(iterations)

rev_ETO = zeros(iterations)
price_ETO = zeros(iterations,N)

rev_RO = zeros(iterations,length(ro_coef_all))
for iter in 1:iterations
    println("Iteration: ", iter)
    # 预设数据（根据你的实际问题替换这些值）
    u = rand(N_u)        # u 向量 ∈ ℝ^N
    A_true = rand(N,N_u)      # a_n ∈ ℝ^N
    # B = rand(N,N)        # b_n ∈ ℝ^N
    # p_dag = round.(rand(N, K),digits=2)  # p_{nk}^† 数据矩阵
    p_dag = [n * k for n in 1:N, k in 1:K]
    B_true = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)

    obj_True,X_True,time_True = Solve_ETO(N,N_u,K,A_true,B_true,u,p_dag)
    rev_True[iter], price_True = compute_oof(X_True, A_true, B_true, u, p_dag)


    A_bias = randn(N,N_u)
    B_bias = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign)
    A_hat = A_true .+ 0.3 .* A_bias
    B_hat = B_true .- 0.3.* B_bias
    obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,u,p_dag)
    rev_ETO[iter], price_ETO[iter,:] = compute_oof(X_ETO, A_true, B_true, u, p_dag)

    ro_index = 1
    for ro_coef in ro_coef_all
        A_lb = A_hat .- ro_coef .* abs.(A_hat)
        A_ub = A_hat .+ ro_coef .* abs.(A_hat)
        B_lb = B_hat .- ro_coef .* abs.(B_hat)
        B_ub = B_hat .+ ro_coef .* abs.(B_hat)
        
        psi_lb = -10.0
        psi_ub = 0.1
        obj_RO,X_RO,time_RO = Solve_RO_Price_with_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,u,p_dag,psi_lb,psi_ub)
        rev_RO[iter,ro_index], price_RO = compute_oof(X_RO, A_true, B_true, u, p_dag)
        ro_index += 1
    end
end

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10


In [22]:
println("rev_True = ",mean(rev_True))
println("rev_ETO = ",mean(rev_ETO))
println("rev_RO = ",mean(rev_RO,dims=1))

rev_True = 8.934732351782987
rev_ETO = 5.0535518920590246
rev_RO = [6.167400233896481 6.173101036349637 5.36562348487501 5.155323195829405 5.369963262897281]


In [23]:
rev_ETO

10-element Vector{Float64}:
 0.32306796966824103
 0.037120978727160116
 0.17040839189500212
 2.7153915986026673
 9.999969626537519
 9.96111367982761
 1.5642141678806372
 5.863691988727748
 9.900667524973437
 9.999872993750214

In [24]:
rev_RO

10×5 Matrix{Float64}:
  0.323068    0.323068   0.323068      0.323068     0.323068
  0.0669351   0.123943   0.205333      0.0669351    0.049339
  5.68763     5.68763    0.000443901   0.00025502   0.00314664
  2.71539     2.71539    2.71539       2.71539      4.8765
  9.99997     9.99997    9.99997       9.99997      9.99997
  9.96111     9.96111    9.96111       9.96111      9.96111
  7.4924      7.4924     7.4924        7.4924       7.4924
  5.52683     5.52683    4.96141       3.99458      3.99458
  9.90067     9.90067    7.99711       6.99952      6.99952
 10.0        10.0       10.0          10.0         10.0